In [1]:
from astropy.io import fits

In [46]:
refheader = fits.getheader("../tests/data/WFPC2u57.fits", ext=1)

In [47]:
refheader

XTENSION= 'TABLE   '           / Ascii table extension                          
BITPIX  =                    8 / 8-bits per 'pixels'                            
NAXIS   =                    2 / Simple 2-D matrix                              
NAXIS1  =                  796 / Number of characters per row                   
NAXIS2  =                    4 / The number of rows                             
PCOUNT  =                    0 / No 'random' parameters                         
GCOUNT  =                    1 / Only one group                                 
TFIELDS =                   49 / Number of fields per row                       
EXTNAME = 'u5780205r_cvt.c0h.tab' / Name of table                               
                                                                                
TTYPE1  = 'CRVAL1  '           /                                                
TBCOL1  =                    1 /                                                
TFORM1  = 'D25.17  '        

In [23]:
testheader = fits.getheader("../tests/data/outputs/header_outtest.fits")

In [24]:
testheader2 = fits.getheader("../tests/data/outputs/header_modify_outtest.fits")

In [27]:
fits.getdata("../tests/data/WFPC2u57.fits")

array([[[-1.5442986 ,  0.91693103, -0.09551175, ...,  0.61729944,
          0.69078577, -0.26579204],
        [-0.8824396 , -1.092421  ,  0.9486601 , ..., -0.15046741,
          0.8917887 , -1.2892787 ],
        [-2.8122025 , -0.19849575, -0.20965841, ...,  1.784664  ,
          0.8454866 , -0.317576  ],
        ...,
        [-2.6957903 , -0.3694622 ,  1.9655508 , ..., -0.2042407 ,
         -0.22842535, -0.3640667 ],
        [-1.8624237 , -0.27407092, -0.05868976, ..., -0.6628461 ,
         -0.37871024, -0.4661588 ],
        [-0.8094051 ,  0.89115345, -0.19425163, ..., -1.342756  ,
         -0.2216233 ,  0.61090004]],

       [[ 0.50445807, -0.17986225, -0.02730826, ...,  0.7757869 ,
         -0.34262717, -0.22135872],
        [ 0.5196407 , -0.27171996, -0.06173214, ..., -0.29152176,
          0.59813404, -0.25505006],
        [ 0.38664252, -0.36898297,  0.92303437, ..., -1.2506361 ,
          0.6241902 , -0.20030949],
        ...,
        [ 0.4626931 , -0.27773476, -0.32133174, ...,  

In [31]:
fits.getdata("../tests/data/outputs/imagemodified.fits", hdu=0)

array([[[-3.0885973 ,  1.8338621 , -0.1910235 , ...,  1.2345989 ,
          1.3815715 , -0.5315841 ],
        [-1.7648792 , -2.184842  ,  1.8973202 , ..., -0.30093482,
          1.7835774 , -2.5785575 ],
        [-5.624405  , -0.3969915 , -0.41931683, ...,  3.569328  ,
          1.6909732 , -0.635152  ],
        ...,
        [-5.3915806 , -0.7389244 ,  3.9311016 , ..., -0.4084814 ,
         -0.4568507 , -0.7281334 ],
        [-3.7248473 , -0.54814184, -0.11737952, ..., -1.3256922 ,
         -0.7574205 , -0.9323176 ],
        [-1.6188102 ,  1.7823069 , -0.38850325, ..., -2.685512  ,
         -0.4432466 ,  1.2218001 ]],

       [[ 1.0089161 , -0.3597245 , -0.05461651, ...,  1.5515738 ,
         -0.68525434, -0.44271743],
        [ 1.0392814 , -0.5434399 , -0.12346429, ..., -0.5830435 ,
          1.1962681 , -0.5101001 ],
        [ 0.77328503, -0.73796594,  1.8460687 , ..., -2.5012722 ,
          1.2483804 , -0.40061897],
        ...,
        [ 0.9253862 , -0.5554695 , -0.6426635 , ...,  

In [37]:
fits.open("../tests/data/outputs/primary_hdu_test.fits")[1].header

IndexError: list index out of range

In [9]:
fits.getheader("../tests/data/0.1_0.1_300_R_swp_splus.fits.fz", ext=1)

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                  300 / length of data axis 1                          
NAXIS2  =                  300 / length of data axis 2                          
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
EQUINOX =        2000.00000000 / Mean equinox                                   
MJD-OBS =   5.837700000000E+04 / Modified Julian date at start                  
RADESYS = 'ICRS    '           / Astrometric system                             
CTYPE1  = 'RA---TAN'           / WCS projection type for this axis              
CUNIT1  = 'deg     '           / Axis unit                                      
CRVAL1  =   0.000000000000E+